<a href="https://colab.research.google.com/github/goteguru/kmooc_python/blob/main/notebooks/en/kmooc_11_1_api_usage_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handling External APIs

We use an API when there's no Python library for something... just kidding, there's a Python library for everything. :-) Still, if there isn't one, or — which is more likely — we don't have the data locally, or our machine's performance isn't sufficient, we have no choice but to use an external system via API calls.

If this is an HTTP-based REST API (there are other kinds too) then the requests library will be our best friend.

Since most REST APIs return a textual format called JSON, let's quickly recap what that is.

## JSON

The JSON format won't be unfamiliar: it's basically (well, almost) like the dict and list structures we already know, containing integers, floats and strings, and even None — except it's called null there. It's easy to use because, of course, there's a library for it.


In [ ]:
import json

# let's create a mixed-data example:
python_structure = {
    "szöveg": "this is a text",
    "számok": [1,2,3,4,5],
    "semmi": None,
    "tuple": (18,42),
    1999: 0.456,
    0: 0
}

# convert to JSON format:
json_text = json.dumps(python_structure)
json_text

'{"sz\\u00f6veg": "ez egy sz\\u00f6veg", "sz\\u00e1mok": [1, 2, 3, 4, 5], "semmi": null, "tuple": [18, 42], "1999": 0.456, "0": 0}'

The resulting text is the JSON. You can see it looks almost exactly like the original Python data, with a few small differences (e.g. None became null, unicode characters were escaped, and keys are always strings).

Why is this useful? JSON is understood by many — many — languages and systems (almost all), so it's an excellent way to transfer structured data.

Because JSON can contain lists and dicts nested within each other, it can describe fairly complex structures.

If we have JSON data and want to work with it in Python, we convert it back:

In [ ]:
python_data = json.loads(json_text)
python_data

{'szöveg': 'ez egy szöveg',
 'számok': [1, 2, 3, 4, 5],
 'semmi': None,
 'tuple': [18, 42],
 '1999': 0.456,
 '0': 0}

It's almost the same as where we started, except the keys became strings and the tuple turned into a list (JSON doesn't know tuples). Unfortunately not every Python structure can be converted to JSON automatically. If we have a more exotic Python structure, e.g. a set, it can't be converted to JSON (automatically).

## API calls: requests

Now that we know what we'll get back, let's look at some examples of how to use these external systems.

Let's start with a currency exchanger that works with current exchange rates. We don't have that information locally, so we access current rates through an API.

#### Currency exchanger

The request yields an HTTP Response object, which always has a status field. (According to the HTTP protocol) a successful response is usually 200-something (mostly 200), and errors are indicated by 400- or 500-something. Check the API documentation for exact values.

If the status code indicates success, we probably received what we expected and can try to parse it into Python data.

In [ ]:
import requests

# send the request
req = requests.get('https://api.fxratesapi.com/latest')

# what's our status code? (200 -> success)
print("status:", req.status_code)

# convert the contained JSON data to Python
rates = req.json()
print("valid:", rates['date'])

rates['base'], rates['rates']['HUF'], "HUF"

status: 200
érvényes: 2025-11-22T19:09:00.000Z


('USD', 332.6800348783, 'HUF')

Modify the code above so that if the status_code is not 200 it does not attempt to display the rates. If you're curious what the entire returned data structure looks like, display the whole rates structure.

#### Universities

Here's another example: what universities are there in Hungary?

In [ ]:
import requests

# send the request
country = "hungary"
# we build the url on the fly here:
url = f'http://universities.hipolabs.com/search?country={country}'
req = requests.get(url)

if (req.status_code == 200):
  data = req.json()
  universities = [row['name'] for row in data]
  print (universities)

If you want to see what the server returns, display the request's text result, or simply paste the URL into your browser's address bar.

Modify the code above so that it only shows the website pages. (If there are multiple, show the first!)


## Generative AI

Let's look at an example of using a pre-written API library (in our case Google Gemini). Because we can't use this library freely, we'll need an API key.

Google's keys can be easily managed (or created) in the key manager (left-side menu, key icon).

`Gemini API keys -> Manage keys in Google AI Studio`

In [ ]:
# never put the API key into source code!
# normally we store it in an environment variable:

# api_key = os.environ['API_KEY']

# in COLAB we can add it to the stored keys
# <---- there's a key icon in the menu on the left
# and access it like this:

from google.colab import userdata
api_key = userdata.get('GEMINI_API_KEY')

39

Once you have the key, just import the appropriate package and use it.

In [ ]:
from google import genai

client = genai.Client(api_key=api_key)

# pick a model
model_info = client.models.get(model="gemini-2.5-flash-lite")

# query some basic information about the selected model:
print(model_info.name)
print(model_info.description )


models/gemini-2.5-flash-lite
Stable version of Gemini 2.5 Flash-Lite, released in July of 2025


In [ ]:
# Ask it something:

response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents='Estimate when will AI overthrow humanity in single sentence.'
)
print(response.text)

AI is unlikely to "overthrow" humanity in a traditional sense, but its growing capabilities may lead to profound societal shifts and challenges that require careful management and adaptation over the coming decades.


It would be similarly straightforward to generate images or videos, but those services are usually paid.

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key=api_key)

response = client.models.generate_images(
    model='imagen-4.0-generate-001',
    prompt='Company logo presenting a purple rabbit holding a pencil',
    config=types.GenerateImagesConfig(
        number_of_images= 1,
    )
)

for generated_image in response.generated_images:
  generated_image.image.show()